In [2]:
import boto3

############# This code block is used for pulling data from S3 using boto3 library
############# Note: Not necessary for this project

#getting relevant information for the S3 bucket

AWS_S3_BUCKET='w210-bucket'
AWS_S3_REGION='us-east-1'
AWS_PROFILE_NAME='default'
session = boto3.Session(profile_name=AWS_PROFILE_NAME)
s3 = session.resource('s3')
s3_client = session.client('s3',region_name=AWS_S3_REGION)
my_bucket = s3.Bucket(AWS_S3_BUCKET)

# #printing all the files in ridership directory 
for objects in my_bucket.objects.filter(Prefix="ridership/"):
     print(objects.key)

# #printing all the files in weather directory 
# for objects in my_bucket.objects.filter(Prefix="weather/"):
#     print(objects.key)

# #reading one file only as pandas df
# obj = s3_client.get_object(Bucket= AWS_S3_BUCKET, Key= "ridership/date-hour-soo-dest-2011.csv") 
# # get object and file (key) from bucket
# initial_df = pd.read_csv(obj['Body'], header=None) 
# initial_df

ridership/date-hour-soo-dest-2011.csv
ridership/date-hour-soo-dest-2012.csv
ridership/date-hour-soo-dest-2013.csv
ridership/date-hour-soo-dest-2014.csv
ridership/date-hour-soo-dest-2015.csv
ridership/date-hour-soo-dest-2016.csv
ridership/date-hour-soo-dest-2017.csv
ridership/date-hour-soo-dest-2018.csv
ridership/date-hour-soo-dest-2019.csv
ridership/date-hour-soo-dest-2020.csv
ridership/date-hour-soo-dest-2021.csv
ridership/date-hour-soo-dest-2022.csv
ridership/df_station_ridership2_$folder$
ridership/first_join.parquet_$folder$
ridership/station_bart.csv
ridership/testt.csv/_SUCCESS
ridership/testt.csv/part-00000-92304ff7-796b-4fd7-9c6f-93d2650b7cf1-c000.csv
ridership/testt.csv/part-00001-92304ff7-796b-4fd7-9c6f-93d2650b7cf1-c000.csv


In [ ]:
import argparse
import csv
import os
import networkx as nx  # type: ignore

DATA_DIR = "/Users/murray/ucb/210/data"

station_names = {
    '12TH': '12th St/Oakland City Center',
    '16TH': '16th St Mission',
    '19TH': '19th St/Oakland',
    '24TH': '24th St Mission',
    'ANTC': 'Antioch',
    'ASHB': 'Ashby',
    'BALB': 'Balboa Park',
    'BAYF': 'Bay Fair',
    'BERY': 'Berryessa',
    'CAST': 'Castro Valley',
    'CIVC': 'Civic Center/UN Plaza',
    'COLM': 'Colma',
    'COLS': 'Coliseum',
    'CONC': 'Concord',
    'DALY': 'Daly City',
    'DBRK': 'Downtown Berkeley',
    'DELN': 'El Cerrito del Norte',
    'DUBL': 'Dublin/Pleasanton',
    'EMBR': 'Embarcadero',
    'FRMT': 'Fremont',
    'FTVL': 'Fruitvale',
    'GLEN': 'Glen Park',
    'HAYW': 'Hayward',
    'LAFY': 'Lafayette',
    'LAKE': 'Lake Merritt',
    'MCAR': 'MacArthur',
    'MLBR': 'Millbrae',
    'MLPT': 'Milpitas',
    'MONT': 'Montgomery St',
    'NBRK': 'North Berkeley',
    'NCON': 'North Concord/Martinez',
    'OAKL': 'Oakland International Airport (OAK)',
    'ORIN': 'Orinda',
    'PCTR': 'Pittsburg Center',
    'PHIL': 'Pleasant Hill/Contra Costa Centre',
    'PITT': 'Pittsburg/Bay Point',
    'PLZA': 'El Cerrito Plaza',
    'POWL': 'Powell St',
    'RICH': 'Richmond',
    'ROCK': 'Rockridge',
    'SANL': 'San Leandro',
    'SBRN': 'San Bruno',
    'SFIA': 'San Francisco International Airport (SFO)',
    'SHAY': 'South Hayward',
    'SSAN': 'South San Francisco',
    'UCTY': 'Union City',
    'WARM': 'Warm Springs/South Fremont',
    'WCRK': 'Walnut Creek',
    'WDUB': 'West Dublin/Pleasanton',
    'WOAK': 'West Oakland'
}

#edges = [('ANTC', 'PCTR'), ('PCTR', 'PITT'), ('PITT', 'NCON'), ('NCON', 'CONC'), ('CONC', 'PHIL'), ('PHIL', 'WCRK'), 
#         'WCRK', 'LAFY', 'ORIN', 'ROCK', 'MCAR', '19TH', '12TH', 'WOAK', 'EMBR', 'MONT', 'POWL', 'CIVC', '16TH', '24TH',
#         'GLEN', 'BALB', 'DALY', 'COLM', 'SSAN', 'SBRN', 'MLBR', 'SFIA'

# Edges from: https://www.bart.gov/system-map

# Yellow line - Antioch - SFO Millbrae
edges = [('ANTC', 'PCTR'), ('PCTR', 'PITT'), ('PITT', 'NCON'), ('NCON', 'CONC'), ('CONC', 'PHIL'), ('PHIL', 'WCRK'), 
         ('WCRK', 'LAFY'), ('LAFY', 'ORIN'), ('ORIN', 'ROCK'), ('ROCK', 'MCAR'), ('MCAR', '19TH'), ('19TH', '12TH'),
         ('12TH', 'WOAK'), ('WOAK', 'EMBR'), ('EMBR', 'MONT'), ('MONT', 'POWL'), ('POWL', 'CIVC'), ('CIVC', '16TH'),
         ('16TH', '24TH'), ('24TH', 'GLEN'), ('GLEN', 'BALB'), ('BALB', 'DALY'), ('DALY', 'COLM'), ('COLM', 'SSAN'),
         ('SSAN', 'SBRN'), ('SBRN', 'SFIA')]

# Red line - Richmond - SFO Millbrae
edges += [('RICH', 'DELN'), ('DELN', 'PLZA'), ('PLZA', 'NBRK'), ('NBRK', 'DBRK'), ('DBRK', 'ASHB'), ('ASHB', 'MCAR'),
          ('SBRN', 'MLBR'), ('MLBR', 'SFIA')]

# Blue line - Dubline/Pleasanton - Daly City
edges += [('DUBL', 'WDUB'), ('WDUB', 'CAST'), ('CAST', 'BAYF'), ('BAYF', 'SANL'), ('SANL', 'COLS'), ('COLS', 'FTVL'),
          ('FTVL', 'LAKE'), ('LAKE', '12TH'), ('LAKE', 'WOAK')]

# Green line - Berryessa/North San Jose - Daly City
edges += [('BERY', 'MLPT'), ('MLPT', 'WARM'), ('WARM', 'FRMT'), ('FRMT', 'UCTY'), ('UCTY', 'SHAY'), ('SHAY', 'HAYW'),
          ('HAYW', 'BAYF')]

# Beige line
edges += [('COLS', 'OAKL')]

#print(edges)
G = nx.Graph()
for s in station_names:
    G.add_node(s)

for e in edges:
    G.add_edge(*e)

#print(G)
#print("Hello")

#foo = nx.shortest_path(G, 'ANTC', 'PITT')
#print(foo)
#print(type(foo))
#bar = nx.shortest_path(G, 'HAYW', 'CONC')
#print(bar)
#print(type(bar))

#baz = nx.shortest_path(G, 'HAYW', 'SFIA')
#print(baz)


parser = argparse.ArgumentParser("parse_daily_bart_trips")
parser.add_argument("input_file", help="The input daily trip CSV file for a year from bart.gov.", type=str)
args = parser.parse_args()

# Now lets access the CSV file, or a sample of one, and add the durations to it.
# Input CSV is of form:
# Date, Hour, Origin station, Destination station, Number of trips.
cnt = 0
lasthour = None
hour = None
with open(args.input_file) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in spamreader:
        cnt += 1
        hour = row[0] + "," + row[1]
        if (lasthour is None):
            weights = {}
            lasthour = hour
        elif hour != lasthour:
#            print("Count: ", cnt, " breaking hour!= lasthour", hour, "!=", lasthour)
            for k, v in sorted(weights.items(), key=lambda item:item[1]):
                print(lasthour, ",", k, ",", v, sep="")
            weights = {}
            lasthour = hour

#        if (cnt > 10):
#            break
        spath = nx.shortest_path(G, row[2], row[3]) # shortest path from origin to destination.
        for station in spath:
            if station in weights:
                weights[station] += int(row[4])  # number of trips form this source/destination pair
            else:
                weights[station] = int(row[4])
            
#        print("From: ", row[2], "To: ", row[3], "Date: ", row[0], "Hour: ", row[1], "Trips: ", row[4])

#        print(', '.join(row))